In [154]:
import cv2
import pandas as pd
import numpy as np

In [161]:
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []

   # if the bounding boxes integers, convert them to floats --
   # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
#  
   # initialize the list of picked indexes   
    pick = []

   # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    x2 = boxes[:,1]
    y1 = boxes[:,2]
    y2 = boxes[:,3]
   # compute the area of  bounding boxes and sort the bounding
   # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

   # keep looping while some indexes still remain in the indexes
   # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
          # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
    
          # find the largest (x, y) coordinates for the start of
          # the bounding box and the smallest (x, y) coordinates
          # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
    
          # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
    
          # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
    
          # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))

   # return only the bounding boxes that were picked using the
   # integer data type
    return boxes[pick].astype("int")

In [162]:
boxes = pd.read_csv('./akn.036.020.left.csv')

In [163]:
rects = np.round(np.asarray(boxes).astype(np.int))[:,1:5]

In [168]:
new = non_max_suppression_fast(rects, 0)

In [169]:
vid = cv2.VideoCapture('./akn.036.020.left.segmented.avi')
ret, frame = vid.read()
for rect in rects:
    frame = cv2.rectangle(frame, (rect[0], rect[2]), (rect[1], rect[3]), (0,255,0))
for rect in new:
    frame = cv2.rectangle(frame, (rect[0], rect[2]), (rect[1], rect[3]), (250,0,0),3)
cv2.imwrite('0.jpg', frame)
vid.release()